In [ ]:
import pandas as pd

main_df = pd.read_csv('2023_Q3.2_AllData_Before_20231215.csv', header=0, index_col=0, low_memory=False).reset_index(drop=True)
print("Original shape: ")
print(main_df.shape)
main_df = main_df.drop_duplicates()

dropnas_df = main_df.dropna(axis=1, how="all")
dropnas_df = dropnas_df.dropna(subset=['Value'])
dropnas_df = dropnas_df.drop(
print("New shape after dropna: ")
print(dropnas_df.shape)

droplist = ['SeriesID','ReleaseStatus', 'ReleaseName', 'isDSDSeries', 'Time_Detail', 
            'Source', 'SeriesObservationCount', 'UpperBound', 'SeriesDescription',
            'LowerBound', 'ObservationID', 'Observation Status', 'FootNote', 'Reporting Type', 
            'GeoAreaCode', 'Cities'
           ]

df = dropnas_df.drop(droplist, axis=1).drop_duplicates()
print("Different after dropping superfluous fields? shape: ")
print(df.shape)


discriminator_df = df.drop(['ID', 'Goal', 'Target', 'Value', 'ValueType', 'TimePeriod', 'BasePeriod', 'GeoAreaName', 'Units'], axis=1)
# discriminator_df['discriminator_combo'] = discriminator_df.apply(lambda row: tuple(row.dropna()), axis=1)
discriminator_df['discriminator_combo'] = discriminator_df.apply(lambda row: tuple((col, val) for col, val in row.dropna().items()), axis=1)
df['discriminator_combo'] = discriminator_df['discriminator_combo']
print("Discriminator DF shape: ")
print(discriminator_df.shape)

conditioning_droplist = ['ID', 'Goal', 'Target', 'Indicator', 'SeriesCode', 'ValueType', 'BasePeriod', 'Age', 'Location', 'Sex', 'Units', 'Level/Status', 'Education level', 'Type of product', 'Name of international institution',
                        'Type of occupation', 'Type of skill', 'Mode of transportation', 'Name of non-communicable disease', 'Type of speed', 'Migratory status', 'Disability status', 'IHR Capacity', 'Quantile', 'Activity',
                         'Policy Domains', 'Policy instruments', 'Sampling Stations', 'Type of waste treatment', 'Grounds of discrimination', 'Parliamentary committees', 'Cause of death', 'Substance use disorders',
                        'Deviation Level', 'Frequency of Chlorophyll-a concentration', 'Food Waste Sector', 'Fiscal intervention stage', 'Level of requirement', 'Type of support', 'Report Ordinal', 'Counterpart', 'Government_Name', 
                        'Severity of price levels', 'Level_of_government', 'Type of renewable technology', 'Population Group', 'Service Attribute', 'Land cover', 'Bioclimatic belt', 'Illicit Financial Flows', 'Nutrient Loading',
                        'Type of OFDI scheme','Nature', ]
unconditioned_df = df.drop(conditioning_droplist, axis=1)
print("Final shape before pivot: ")
print(unconditioned_df.shape)

conditioned_df = unconditioned_df.pivot_table(index=['GeoAreaName', 'TimePeriod'],
                                             columns='discriminator_combo',
                                             values='Value',
                                             aggfunc='mean').reset_index()

print("Total number of values in conditioned DF: ")
print(conditioned_df.count().sum()) 

conditioned_df.to_csv("2d SDG Dataset.csv", index=False)

In [ ]:
unconditioned_df[pd.to_numeric(unconditioned_df['Value'], errors='coerce').isna()]

In [ ]:
conditioned_df.count().sum()

In [ ]:
main_df.Value.isna().sum()

In [ ]:
conditioned_df.count().sort_values()

In [ ]:
unconditioned_df[unconditioned_df.discriminator_combo == ("15.4.2", "ER_MTN_GRNCVI", "ARTIFICIAL", "NIVAL")]

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df.Value.isna().sum()